In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
block_size = 64
batches = 32
train_loop = 5000
learning_rate = 5e-4
train_split = 0.8
debug_loop = 500
embeds = 512
heads = 4
layers = 8
dropout = 0.2

cuda


In [2]:
chars = ""
with open('persion.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '"', '.', '0', '1', '2', '3', '4', '5', '7', 'A', 'B', 'H', 'I', 'L', 'M', 'P', 'W', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
45


In [3]:
str2int = { ch:i for i,ch in enumerate(chars) }
int2str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [ str2int[c] for c in s ]
decode = lambda l: ''.join([ int2str[i] for i in l ])
data = torch.tensor(encode(text), dtype=torch.long).to(device)
print(data[:100])

tensor([39, 37, 23, 36,  1,  5,  1, 34, 23, 36, 37, 33, 32, 19, 37,  0,  2, 14,
         1, 28, 39, 37, 38,  1, 20, 33, 39, 25, 26, 38,  1, 19,  1, 20, 36, 19,
        32, 22,  1, 32, 23, 41,  1, 26, 33, 39, 37, 23,  3,  0, 14,  1, 30, 27,
        29, 23,  1, 38, 33,  1, 22, 19, 32, 21, 23,  1, 19, 38,  1, 38, 26, 23,
         1, 21, 30, 39, 20,  3,  0, 14,  1, 36, 39, 32,  1, 19,  1, 22, 33, 25,
         1, 33, 20, 23, 22, 27, 23, 32, 21, 23], device='cuda:0')


In [4]:
n = int(train_split*len(data))
train = data[:n]
valid = data[n:]

def get_batch(split):
    data = train if split=='train' else valid
    ix = torch.randint(len(data) - block_size, (batches,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[19, 40, 33,  ..., 21, 33, 31],
        [19, 27, 37,  ..., 23,  1, 32],
        [ 1, 27, 32,  ..., 31,  1, 19],
        ...,
        [37, 23, 19,  ..., 27, 37, 38],
        [ 0, 14,  1,  ..., 23, 23,  1],
        [ 1, 38, 33,  ...,  1, 24, 39]], device='cuda:0')
target:
tensor([[40, 33, 36,  ..., 33, 31, 34],
        [27, 37, 23,  ...,  1, 32, 33],
        [27, 32,  1,  ...,  1, 19,  1],
        ...,
        [23, 19,  3,  ..., 37, 38, 23],
        [14,  1, 30,  ..., 23,  1, 26],
        [38, 33,  1,  ..., 24, 39, 36]], device='cuda:0')


In [5]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embeds, head_size, bias=False)
        self.query = nn.Linear(embeds, head_size, bias=False)
        self.value = nn.Linear(embeds, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-.5
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embeds)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embeds, 4*embeds),
            nn.ReLU(),
            nn.Linear(4*embeds, embeds),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, embeds, heads):
        super().__init__()
        head_size = embeds // heads
        self.sa = MultiHeadAttention(heads, head_size)
        self.ffwd = FeedForward(embeds)
        self.ln1 = nn.LayerNorm(embeds)
        self.ln2 = nn.LayerNorm(embeds)
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class Kiwi(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embeds)
        self.position_embedding = nn.Embedding(block_size, embeds)
        self.blocks = nn.Sequential(*[Block(embeds, heads=heads) for _ in range(layers)])
        self.final_norm = nn.LayerNorm(embeds)
        self.head = nn.Linear(embeds, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        tok = self.token_embedding(index)
        pos = self.position_embedding(torch.arange(T, device=device))
        x = tok + pos
        x = self.blocks(x)
        x = self.final_norm(x)
        logits = self.head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        for iter in range(max_new_tokens):
            index_crop = index[:, -block_size:]
            logits, loss = self.forward(index_crop)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            print(f"\r<{'*' * int(10 * iter/max_new_tokens)}{' ' * (10 - int(10*iter/max_new_tokens))}>", end='', flush=False)
        print("\r<**********>")
        return index

m = Kiwi(vocab_size)
model = Kiwi(vocab_size).to(device)

In [6]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(debug_loop)
        for k in range(debug_loop):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
%%time
optim = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(train_loop):
    if iter % debug_loop == 0:
        losses = estimate_loss(model)
        print(f'step: {iter}, train loss: {losses['train']:.6f}, valid loss: {losses['val']:.6f}')

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()
print(loss.item())

step: 0, train loss: 3.918626, valid loss: 3.920266
step: 500, train loss: 0.612665, valid loss: 0.636477
step: 1000, train loss: 0.351458, valid loss: 0.367271
step: 1500, train loss: 0.309351, valid loss: 0.331282
step: 2000, train loss: 0.285331, valid loss: 0.308199
step: 2500, train loss: 0.262747, valid loss: 0.295625
step: 3000, train loss: 0.247159, valid loss: 0.277548
step: 3500, train loss: 0.241433, valid loss: 0.273690
step: 4000, train loss: 0.230591, valid loss: 0.264746
step: 4500, train loss: 0.226760, valid loss: 0.263119
0.2556634545326233
CPU times: total: 5min 15s
Wall time: 14min 57s


In [8]:
%%time
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(model.generate(ctx, max_new_tokens=1000)[0].tolist())
print(generated)

<**********>

I enjoy exercising for long hours.
I spend my free time reading plast yor ago."
"I love watching sports on tv.
I like to paint in my spare time.
I live in ohio."
"I have a music.
I am a ser 22 year old girl.
I am taking cing civil war antiques.
I have 3 sisters."
"I love to meut new people.
My favorite singer is time of missississippi.
I am 34 years old trans boy."
"I love dogs but hate cats.
I love to play on second life.
I am a huge ed sheeran fan.
I am looking forward to retiring in five years.
I like ot play racquetball.
I love tryink ansas.
I love to build things with my hands.
I have dinner with my family and then go out play with friends."
"I love to surf and skate.
I have two cats.
I like children in that are in their kindergarten."
"I am colorblind."
"My best friend is a robot.
I am color blind and live with my little sister."
"I have a husband works nights of america.
I am a student.
I have a dog named maxed max.
I am serving in south korea.
I am a violent perso

In [9]:
prompt = 'Hi. We are the developers of Kiwi'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=500)[0].tolist())
print(generated_chars)

KeyError: 'K'

In [ ]:
'''
import speech_recognition as sr
import pyttsx3

gender = 1

listener = sr.Recognizer()
engine = pyttsx3.init('sapi5')
voices = engine.getProperty("voices")
engine.setProperty('voice',voices[gender].id)

def talk(audio):
    engine.say(audio)
    engine.runAndWait()

talk(generated)
'''